In [8]:
import gensim
import nltk
import pandas as pd

from gensim import corpora
from gensim.summarization import keywords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob

from modules import CBR

pd.set_option("display.width", None)
import warnings

warnings.filterwarnings("ignore")
pd.options.display.max_columns = None

# Load the data as a Pandas dataframe
df = pd.read_csv("yelp_dataset/non_food_merged.csv")
df = df.drop(df.columns[0], axis=1)
my_city = "Brentwood".lower()
dest_city = "Philadelphia".lower()
df["name"] = df["name"].str.lower()
df["city"] = df["city"].str.lower()
df.head(10)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,BusinessAcceptsCreditCards,WheelchairAccessible,BusinessParking,ByAppointmentOnly,review_id,user_id,stars_y,useful,funny,cool,text,date
0,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},e_PZZ0m2sEG9UovGRxdZRQ,dT6O_rV9DWYS-zHXhA6S6w,4,3,2,2,This has become my go to place for shoes. I a...,2015-12-06 16:46:43
1,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},WNv6UCHTmce7wgImLKP4sg,AAYvaNRQ0TD_2Lpo-wFOUA,4,3,2,1,"Oh, I do enjoy Famous Footwear. \n\nThe occasi...",2016-03-20 21:52:13
2,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},VwoJCaULB5cRGnDiYGFYJA,V9fW3-fJ-sEMz_ewPpzXXg,1,2,0,0,Ordered shoes online it clearly says free retu...,2014-10-03 21:42:17
3,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},crAIe0dciujX2sFvi7bSkA,iSgusF1eKu23mNG87zav4Q,1,0,0,0,Poor customer service will never be shopping a...,2018-08-13 15:40:01
4,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},DRvrkDpdTOXK4j-Vx9qyDg,XfdP4UU3xMcdJbM3qUIaPA,5,0,0,0,Found a great deal on a pair of Nike running s...,2017-01-25 19:00:25
5,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},fE2AyAeOCJWBc0dTWpv4kA,eTvp_hYnsrI5-ow_sQ31_g,4,3,0,1,"I was in here a few months back, and only foun...",2011-09-19 17:30:26
6,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},VbsP--bypefXTeXcdfmC1w,lfR20JWTH2_rTOMj7N6Bng,1,3,1,0,Pretty unprofessional when the workers are sit...,2016-02-20 02:58:44
7,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},0Q4r8IeRXhtySU46dS6_Mw,bYENop4BuQepBjM1-BI3fA,3,5,3,

In [9]:
tst = df.iloc[:20000,]
tmp = CBR.sentiment_analysis(tst)
tst_lda, tst_dict = CBR.lda_model(tst, num_topics=10, remove_stopwords=True)
tmp = CBR.extract_topics(tmp, tst_lda, tst_dict, num_words=10, remove_stopwords=True)
tmp = CBR.extract_keywords(tmp, remove_stopwords=True)
tmp

[nltk_data] Downloading package omw-1.4 to /Users/yhkim/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /Users/yhkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,BusinessAcceptsCreditCards,WheelchairAccessible,BusinessParking,ByAppointmentOnly,review_id,user_id,stars_y,useful,funny,cool,text,date,sentiment_polarity,sentiment_subjectivity,topic_id,topic_words,keywords
0,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},e_PZZ0m2sEG9UovGRxdZRQ,dT6O_rV9DWYS-zHXhA6S6w,4,3,2,2,This has become my go to place for shoes. I a...,2015-12-06 16:46:43,0.109394,0.519021,1,"[store, shop, selection, find, items, store., ...","[become, go, place, shoes, rewards, member, of..."
1,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},WNv6UCHTmce7wgImLKP4sg,AAYvaNRQ0TD_2Lpo-wFOUA,4,3,2,1,"Oh, I do enjoy Famous Footwear. \n\nThe occasi...",2016-03-20 21:52:13,0.109394,0.519021,2,"[place, great, like, get, good, one, really, -...","[enjoy, famous, footwear, oh, occasional, bogo..."
2,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},VwoJCaULB5cRGnDiYGFYJA,V9fW3-fJ-sEMz_ewPpzXXg,1,2,0,0,Ordered shoes online it clearly says free retu...,2014-10-03 21:42:17,0.109394,0.519021,8,"[would, get, one, told, back, said, time, coul...","[ordered, shoes, online, clearly, says, free, ..."
3,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},crAIe0dciujX2sFvi7bSkA,iSgusF1eKu23mNG87zav4Q,1,0,0,0,Poor customer service will never be shopping a...,2018-08-13 15:40:01,0.109394,0.519021,8,"[would, get, one, told, back, said, time, coul...","[poor, customer, service, never, shopping, sto..."
4,n_0UpQx1hsNbnPUSlodU8w,famous footwear,"8522 Eager Road, Dierbergs Brentwood Point",brentwood,MO,63144,38.627695,-90.340465,2.5,13,1,"{'RestaurantsPriceRange2': '2', 'BikeParking':...","Sporting Goods, Fashion, Shoe Stores, Shopping...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-18:0', ...",{},{},"{'garage': False, 'street': False, 'validated'...",{},DRvrkDpdTOXK4j-Vx9qyDg,XfdP4UU3xMcdJbM3qUIaPA,5,0,0,0,Found a great deal on a pair of Nike running s...,2017-01-25 19:00:25,0.109394,0.519021,1,"[store, shop, selection, find, items, store., ...","[found, great, deal, pair, nike, running, shoe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,FSiPq3GSzHOch1axV8VCKA,blind dog tavern,"50 N Sierra St, Ste 1A",reno,NV,89501,39.525276,-119.813913,4.5,59,1,"{'RestaurantsReservations': 'False', 'Restaura...","Nightlife, Bars, Speakeasies, Lounges, Dive Ba...","{'Monday': '15:0-2:0', 'Tuesday': '15:0-2:0', ...",{},True,"{'garage': False, 'street': True, 'validated':...",False,IeIkt1BY9zKi4p3HM-tcJg,TzjQHqlWUcWkQ1zLavRvig,5,0,0,0,"Seriously, my new favorite place in Reno. My b...",2019-03-16 03:40:28,0.291484,0.561843,2,"[place, great, like, get, good, one, really, -...","[

In [10]:
tmp_agg = CBR.content_based_aggregate(tmp)
tmp_agg

,business_id,name,city,state,stars,review_count,sentiment_polarity,sentiment_subjectivity,topic_words_top20,keywords_top20,reviews
0,-7GjicSH_rM8JeZGCXGcUg,double decker,tampa,FL,3.181818,86.0,0.175360,0.499188,"[get, one, place, great, like, good, really, -...","[bar, karaoke, n, place, fun, great, like, nig...",We stopped in while bar hopping for a friend's...
1,-9n0NDe_pP1ZnrWr-lsDXQ,ralph's barber shop,glenside,PA,3.611111,18.0,0.170614,0.521920,"[get, place, time, great, hair, I've, always, ...","[haircut, great, get, hair, barber, n, place, ...",An incredible local family establishment. I've...
2,-BhSR6dAry5-2x3ndjX_9w,meister's barber shop,philadelphia,PA,3.913580,79.0,0.231719,0.538825,"[get, time, place, great, hair, I've, always, ...","[cut, great, hair, n, haircut, barber, place, ...",Coming from Los Angeles where the hair style c...
3,-UNPalKlpI-_2ejgFNCBPg,francesca's,nashville,TN,3.583333,12.0,0.189286,0.586814,"[store, shop, selection, find, items, store., ...","[store, n, like, cute, back, one, little, --, ...",This store is packed into a pretty small space...
4,-cK2OGOzkvSaxkb91SIjVg,fantastic sams cut & color,zephyrhills,FL,2.333333,6.0,0.181969,0.525376,"[time, get, hair, great, I've, always, cut, re...","[get, cut, appointment, n, hair, one, first, t...",Fantastic Sam's Hair Salon in Zephyrhills...
...,...,...,...,...,...,...,...,...,...,...,...
846,zn-WXkqHag5FSVc_LC9dEQ,renee hair salon,havertown,PA,4.200000,5.0,0.254306,0.594231,"[time, get, hair, great, I've, always, cut, re...","[salon, hair, renee, told, elena, amy, amazing...",I've been going to Renee for 16 years & I woul...
847,znTKlh4x8NoBIojm4Yo5hA,albert's transportation,mount laurel,NJ,3.850000,20.0,0.209416,0.499913,"[get, time, one, would, told, back, said, coul...","[albert, back, service, wedding, time, pick, d...","This is my first ever yelp review, because I t..."
848,znwTYnVgJ1MKQguvqEtMrA,"the foot whisperer, inc",eagle,ID,4.555556,9.0,0.361408,0.561340,"[get, place, time, great, one, hair, I've, alw...","[jeanne, pedicure, best, experience, spa, rela...",I've left voicemail messages with this busines...
849,zyge4T5eSiPHq1-IaJb_Qg,nancy le nails,philadelphia,PA,1.800000,10.0,0.076507,0.483948,"[get, time, would, one, told, back, said, coul...","[n, service, nails, like, get, went, salon, go...",Came here for my first time with my friend and...


In [13]:
cs_tmp = CBR.calc_cosine_similarity(
    tmp_agg, "famous footwear", "brentwood", "philadelphia"
)
CBR.content_based_recommender(cs_tmp)

,business_id,name,city,state,sentiment_polarity,sentiment_subjectivity,topic_words_top20,keywords_top20,reviews,sentiment_polarity_rank,sentiment_subjectivity_rank,topic_words_top20_rank,keywords_top20_rank,reviews_rank,avg_rank
0,IAj1Lw3FAOY-yZn4IO7ElQ,baum's dancewear,philadelphia,PA,1.0,1.0,1.000000,0.389490,0.0,1,1,1,1,1,1.0
1,he6ypFmnUF95PlNUWT6i5g,tj maxx,philadelphia,PA,1.0,1.0,1.000000,0.270369,0.0,1,1,1,3,1,1.4
2,LvFmVnPSbi0lmgpcK_qw-Q,foot locker,philadelphia,PA,1.0,1.0,0.894737,0.324443,0.0,1,1,3,2,1,1.6
3,9VRg8Ho9SoZWKPmjfrpVmw,forman mills,philadelphia,PA,1.0,1.0,0.945905,0.216295,0.0,1,1,2,5,1,2.0
4,eJ77e9lGxY3ArzaoDbHhYw,paddy whacks irish sports pub - south street,philadelphia,PA,1.0,1.0,0.894737,0.216295,0.0,1,1,3,5,1,2.2
5,C2KhibuDzv3HJvZubkHTJA,village thrift stores,philadelphia,PA,1.0,1.0,1.000000,0.166924,0.0,1,1,1,7,1,2.2
6,tzVLTQNUtiCAxUZmEuF0vQ,1 tippling place,philadelphia,PA,1.0,1.0,0.894737,0.166924,0.0,1,1,3,7,1,2.6
7,rm2XUoqkJn-d5gByPUwamw,eyesite,philadelphia,PA,1.0,1.0,0.718185,0.210526,0.0,1,1,4,6,1,2.6
8,-fqjV6QqrKLUxpYCJoSsOQ,gamestop,philadelphia,PA,1.0,1.0,0.894737,0.157895,0.0,1,1,3,9,1,3.0
9,9Y0jUKfeDFJUStlVHOlc9Q,tie bar,philadelphia,PA,1.0,1.0,0.718185,0.162221,0.0,1,1,4,8,1,3.0
